In [24]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [25]:
# Load the uploaded CSV file
file_path = '/Users/user/Desktop/python-tasks/test-week5/MOCK_DATA.csv'

# This reads the CSV file and loads only 75% of the rows
df = pd.read_csv(file_path)
rows_to_load = int(len(df) * 0.75)
df = df.iloc[:rows_to_load]

# Display the first few rows to understand the structure
df.head()

,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29.000000000
1,1029,China,47,2016-01-08 03:13:12.000000000
2,1025,Nigeria,10,2023-02-20 19:06:14.000000000
3,1006,United Kingdom,48,2022-05-26 12:46:35.000000000
4,1024,Philippines,16,2016-12-13 03:44:04.000000000


In [31]:
# Remove consecutive duplicate rows using the `drop_duplicates` method with 'keep' set to 'first'
df = df.drop_duplicates(keep='first')

# Convert "Sign-up Date" to datetime, with errors='coerce' to handle invalid formats
df['Sign-up Date'] = pd.to_datetime(df['Sign-up Date'], errors='coerce')

# Replace dates from the year 2020 with NaN
df.loc[df['Sign-up Date'].dt.year == 2020, 'Sign-up Date'] = np.nan

# Drop rows where "Total Purchases" is less than 5 and the customer is from Canada
df = df[~((df['Total Purchases'] < 5) & (df['Country'] == 'Canada'))]

# Display the cleaned data
df.head()

,Customer ID,Country,Total Purchases,Sign-up Date,Loyalty Score
0,1015,Palestinian Territory,12,2022-04-03 21:42:29,2
1,1029,China,47,2016-01-08 03:13:12,3
2,1025,Nigeria,10,2023-02-20 19:06:14,2
3,1006,United Kingdom,48,2022-05-26 12:46:35,3
4,1024,Philippines,16,2016-12-13 03:44:04,2


In [28]:
# Calculate membership duration in years from the current date
current_date = datetime.now()
df['Membership Duration'] = (current_date - df['Sign-up Date']).dt.days / 365.25

# This defines the loyalty score calculation function
def calculate_loyalty_score(row):
    if row['Total Purchases'] > 20:
        return 3
    elif 10 <= row['Total Purchases'] <= 20:
        return 2
    elif row['Total Purchases'] < 10:
        # Check if the customer has been a member for more than 3 years
        if row['Membership Duration'] > 3:
            return 2
        else:
            return 1

# Apply the function to calculate the loyalty score
df['Loyalty Score'] = df.apply(calculate_loyalty_score, axis=1)

# Drop the temporary 'Membership Duration' column as it's no longer needed
df = df.drop(columns=['Membership Duration'])

# Display the data with the new 'Loyalty Score' column
df.head()

,Customer ID,Country,Total Purchases,Sign-up Date,Loyalty Score
0,1015,Palestinian Territory,12,2022-04-03 21:42:29,2
1,1029,China,47,2016-01-08 03:13:12,3
2,1025,Nigeria,10,2023-02-20 19:06:14,2
3,1006,United Kingdom,48,2022-05-26 12:46:35,3
4,1024,Philippines,16,2016-12-13 03:44:04,2


In [29]:
# Exclude Mexico from the analysis
df_filtered = df[df['Country'] != 'Mexico']

# Group by "Country" and calculate average and maximum "Total Purchases"
aggregated_data = df_filtered.groupby('Country').agg(
    Average_Purchases=('Total Purchases', 'mean'),
    Total_Purchases=('Total Purchases', 'max'),
    Most_Common_Loyalty_Score=('Loyalty Score', lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

# To display the aggregated data
aggregated_data.head()

,Country,Average_Purchases,Total_Purchases,Most_Common_Loyalty_Score
0,Afghanistan,8.000000,20,2
1,Albania,20.500000,38,2
2,Andorra,11.000000,11,2
3,Angola,15.000000,29,2
4,Argentina,24.692308,45,3


In [30]:
# Get today's date and format it as DD-MM-YYYY
today_date = datetime.now().strftime("%d_%m_%Y")
file_name = f"processed_data_{today_date}.csv"

# Seleect the specified columns and shuffle the DataFrame before displaying
final_df = df[['Customer ID', 'Loyalty Score', 'Country', 'Total Purchases']]


# Shuffle the DataFrame and print the first 10 rows
shuffled_df = final_df.sample(frac=1).head(10)
print(shuffled_df)


# Specify the output directory
output_dir = "/Users/user/Desktop/python-tasks/test-week5"

# Check if the directory exists or create it
if not os.path.exists(output_dir):
    print(f"Directory does not exist. Creating: {output_dir}")
    os.makedirs(output_dir)

# Full file path
file_path = os.path.join(output_dir, file_name)
print(f"Saving file to: {file_path}")

# Save the final processed DataFrame to a CSV file
final_df.to_csv(file_path, index=False)


     Customer ID  Loyalty Score      Country  Total Purchases
695         1026              3        China               24
273         1016              3       Canada               33
81          1012              1       France                7
224         1004              3       Norway               32
225         1027              2    Indonesia                5
40          1016              3     Colombia               35
307         1018              3     Colombia               43
317         1029              3       Russia               45
605         1025              3  Philippines               23
328         1005              2       Russia                9
Saving file to: /Users/user/Desktop/python-tasks/test-week5/processed_data_21_03_2025.csv


- The dataset was successfully loaded, and only the first 75% of rows were used as instructed.
- Consecutive duplicate rows were removed efficiently to maintain data quality.
- The 'Sign-up Date' column was converted to datetime format, and entries from the year 2020 were correctly replaced with NaT values.
- Rows where the 'Total Purchases' were less than 5 and the customer was from Canada were dropped as specified.
- A new 'Loyalty Score' column was created based on the number of purchases and membership duration, adhering to the given rules.
- Data aggregation was performed to calculate the average and maximum purchases per country, as well as the most common loyalty score. Notably, data from Mexico was excluded from this analysis.
- The final processed DataFrame was saved with the required format, and a sample of 10 shuffled rows was displayed.